In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD, Adadelta
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model,load_model
import sys
import os

from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.framework import ops
from tensorflow.python.training import optimizer
import tensorflow as tf


import numpy as np
import random
import math
import matplotlib.pyplot as plt


In [3]:
#Config
epochs = 400
batch = 128
data_size = 10000

def trainingData():    
    def target(x):
    	assert( x >= 0 )
    	y = x * math.sin(5*math.pi *x)
    	return y
    
    x = []
    y = []
    for i in range(data_size):
        x.append(random.random())        
    y = [target(i) for i in x]
    
    return x,y

def createModel():
    model = Sequential()
    model.add(Dense(10, input_shape=(1,), activation='relu'))
    model.add(Dense(18,  activation='relu'))
    model.add(Dense(15,  activation='relu'))
    model.add(Dense(4,  activation='relu'))
    model.add(Dense(1,  activation='linear'))    
    
    optimizer = SGD(lr=0.11, momentum=0.9, decay=1e-8, nesterov=False)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  )
    return model

def get_weight_grad(model, inputs, outputs):
    """ Gets gradient of model for given inputs and outputs for all weights"""
    grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    return output_grad



def compressed_weights(model):
    model_weights = np.array(model.get_weights())
    #print(DNN_weights)
    comp = []
    for i in range(len(model_weights)):
        model_weights[i] = model_weights[i].reshape(len(model_weights[i]),-1)
        model_weights[i] = model_weights[i].flatten()
        model_weights[i] = model_weights[i].reshape(len(model_weights[i]),-1)
        #print(model_weights[i].shape)
        for j in range(len(model_weights[i])):
            comp.append(model_weights[i][j])
    comp = np.array(comp).flatten()
    return comp
    

def get_gradients_norm(model, inputs, outputs):
    #from https://stackoverflow.com/questions/51140950/how-to-obtain-the-gradients-in-keras
    """ Gets gradient of model for given inputs and outputs for all weights"""
    grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    grad_sum = 0
    for i in range(len(output_grad)):
        grad_sum += np.sum(output_grad[i])**2
    grad_norm = grad_sum ** 0.5
    return grad_norm


#Ref:
#https://stackoverflow.com/questions/46473823/how-to-get-weight-matrix-of-one-layer-at-every-epoch-in-lstm-model-based-on-kera
#https://github.com/keras-team/keras/issues/2231
class CollectWeightCallback(Callback):
    def __init__(self, layer_index):
        super(CollectWeightCallback, self).__init__()
        self.layer_index = layer_index
        self.weights = []
        
    
    def on_train_begin(self, logs=None):
        self.epochs = 0
        self.train_batch_loss = []

    def on_epoch_end(self, epoch, logs=None):        
        self.train_batch_loss.append(logs.get('loss'))
        if self.epochs % 3 ==0:
            layer = self.model.layers[self.layer_index]
            self.weights.append(layer.get_weights())
            
        self.epochs +=1   
        

In [6]:
def minimal_ratio(model, sample_amount, x, y, has_metrics=False):
	max_len = 1e-5
	weights = np.array(model.get_weights())
	config = model.get_config()
	pmodel = Model.from_config(config)
	if has_metrics:
		base_loss, metrics = model.evaluate(x=x, y=y)
	else:
		base_loss = model.evaluate(x=x, y=y)

	num_greater = 0

	for i in range(sample_amount):
		noise = np.random.rand(weights.size)
		noise_len = np.random.uniform(0.0, max_len)
		noise *=  (noise_len / np.linalg.norm(noise))
		noise.reshape(weights.shape)

		sample = weights + noise
		sample = [ a for a in sample ]

		pmodel.set_weight(sample)
		if has_metrics:
			loss, metrics = pmodel.evaluate(x=x, y=y)
		else:
			loss = pmodel.evaluate(x=x, y=y)
		if loss > base_loss:
			num_greater += 1
	return num_greater / sample_amount

In [10]:
if __name__ =='__main__':
    model = load_model("model0_0")
    x, y = trainingData()
    minimal_ratio(model, 100, x, y, has_metrics=False)

KeyError: 'name'